### Input

In [ ]:
#input result output file and How many training results are there
dp_or_others = "others"
filename = r"G:\My Drive\PhD\Dr. Daphne Yao\Mimic Project\August_2021_error_bar\2 train_output\sharminafrose\multiple_original.txt"
HowManyTrainingResultAreHere = 3
HowManyEpoch = 100

### PRC max values 

In [ ]:
#needed
with open(filename,'r') as f:
    targets_PRC = [s for s in f.readlines() if "AUC of PRC" in s]
    
listofPRC = []
for line in targets_PRC:
    listofPRC.append(float(line[13:18]))
len(listofPRC)


### Model Names: models

In [ ]:
# needed
with open(filename,'r') as f:
    targets_Model = [s for s in f.readlines() if "/keras_states/" in s]
    
listofModel = []
for line in targets_Model:
    listofModel.append(line)
len(listofModel)

### Model Names: Balanced Accuracy

In [ ]:
#needed
with open(filename,'r') as f:
    targets_Recall_0 = [s for s in f.readlines() if "recall class 0" in s]
    
listofRec0 = []
for line in targets_Recall_0:
    listofRec0.append(float(line[17:22]))
len(listofRec0)

with open(filename,'r') as f:
    targets_Recall_1 = [s for s in f.readlines() if "recall class 1" in s]
    
listofRec1 = []
for line in targets_Recall_1:
    listofRec1.append(float(line[17:22]))
len(listofRec1)


In [ ]:
# needed
balancedAccuracy = []
for i in range(len(listofRec1)):
    balacc = (listofRec1[i] + listofRec0[i])/2
    balancedAccuracy.append(balacc) 

len(balancedAccuracy)
    

In [ ]:
#calculation for graph
import heapq

upperval = HowManyEpoch*2
lowerval= 1
bestprcindexfinal = []
for ran in range(HowManyTrainingResultAreHere):
    graph_prc = []
    graph_bal_acc = []
    for i in range(lowerval,upperval,2):
            graph_prc.append(listofPRC[i])
            graph_bal_acc.append(balancedAccuracy[i])
            
    print("\n*******information about", str(ran + 2) , "unit run *********")
    list_index = heapq.nlargest(3, range(len(graph_bal_acc)), key=graph_bal_acc.__getitem__)
    print(list_index)
    maximum_index = max(list_index)
    minimum_index = min(list_index)
    print(heapq.nlargest(3, graph_bal_acc))
    
    modified_Prc_List = graph_prc[minimum_index:maximum_index]
    highestprc_idx = modified_Prc_List.index(max(modified_Prc_List))
    original = highestprc_idx + minimum_index
    print((original+1), " epoch has highest prc value of " + str(max(modified_Prc_List)))
    bestprcindexfinal.append(original)
    lowerval = lowerval + HowManyEpoch*2
    upperval = upperval + HowManyEpoch*2


In [ ]:
#considered epoch (+1 when read as index start with 0)
bestprcindexfinal

In [ ]:
# model -- prints which epoch to consider; test commands to execute and other instructions
final_models = []

for ran in range(HowManyTrainingResultAreHere):
    final_models.append(listofModel[ran*HowManyEpoch + bestprcindexfinal[ran]])

print(final_models)
for mod in final_models:
    print("python -um mimic3models.in_hospital_mortality.main --network mimic3models/keras_models/lstm.py --dim 16 --depth 2 --batch_size 8 --dropout 0.3 --timestep 1.0 --load_state mimic3models/in_hospital_mortality/keras_states/{} --mode test".format(str(mod[77:])))
    #print("python -um mimic3models.decompensation.main --network mimic3models/keras_models/lstm.py --dim 128 --depth 1 --batch_size 8 --timestep 1.0 --load_state mimic3models/decompensation/keras_states/{} --mode test".format(str(mod[70:])))
#model_name = 'dp_age4050_2'
model_name = 'decomp_dp_age30_1'
val_or_test = 'val'
print("mkdir csv_files")
print("mkdir csv_files/" + model_name)
print("mkdir csv_files/" + model_name + "/"+ val_or_test)
i=2
for mod in final_models:
    if dp_or_others == "others":
        print("mv test_predictions/" + str(mod[70:])+ ".csv " + "csv_files/" + model_name + "/"+ val_or_test)
    else:
        print("mv test_predictions/" + str(mod[70:])+ ".csv " + "csv_files/" + model_name + "/"+ val_or_test + "/unit_" + str(i)+ '.csv')
        i = i+1

    
